Excel → JSON Converter (fault primitives v0.2)
This .py mirrors the logic included in the Jupyter notebook:
  /mnt/data/xlsx_to_fault_json.ipynb

Usage (example):
  python xlsx_to_fault_json.py /mnt/data/fault_with_S.xlsx /mnt/data/fault_primitives.json

In [2]:
import json
import os
import re
import sys
from typing import Any, Dict, List, Optional

import pandas as pd

# normalize_binary_cell
### 函數目標
`normalize_binary_cell` 函數的目標是將輸入值標準化為二進制值（0 或 1），或者返回 `None` 表示無效值。該函數主要用於處理可能包含空值、無效數據或格式不一致的輸入。

### 函數輸入
- `value`：可以是任何類型的輸入值，包括 `None`、數字（整數或浮點數）、字符串等。

### 函數輸出
- 返回 `0` 或 `1`：如果輸入值可以成功轉換為二進制值。
- 返回 `None`：如果輸入值為空、無效或無法轉換為二進制值。

### 執行過程
1. **檢查空值**：
    - 如果輸入值為 `None`，直接返回 `None`。
    - 如果輸入值是浮點數且為 `NaN`，返回 `None`。

2. **處理字符串輸入**：
    - 去除字符串首尾的空格。
    - 如果字符串為空，返回 `None`。
    - 如果字符串為 "0" 或 "1"，轉換為整數並返回。
    - 嘗試將字符串轉換為整數，僅當結果為 0 或 1 時返回，否則返回 `None`。

3. **處理數字輸入**：
    - 如果輸入值是整數或浮點數，嘗試將其轉換為整數。
    - 僅當結果為 0 或 1 時返回，否則返回 `None`。

4. **其他類型輸入**：
    - 如果輸入值不屬於上述類型，返回 `None`。

### 測試數據
以下是測試數據及其對應的輸出：
- `None` → `None`
- `float('nan')` → `None`
- `""` → `None`
- `"0"` → `0`
- `"1"` → `1`
- `"  1  "` → `1`
- `"2"` → `None`
- `"abc"` → `None`
- `0` → `0`
- `1` → `1`
- `2` → `None`
- `0.0` → `0`
- `1.0` → `1`
- `2.0` → `None`

In [4]:
def normalize_binary_cell(value: Any) -> Optional[int]:
    """(See notebook for full docstring)"""
    if value is None:
        return None
    try:
        import math
        if isinstance(value, float) and math.isnan(value):
            return None
    except Exception:
        pass
    if isinstance(value, str):
        s = value.strip()
        if s == "":
            return None
        if s in {"0","1"}:
            return int(s)
        try:
            num = int(s)
            return num if num in (0,1) else None
        except Exception:
            return None
    if isinstance(value, (int,float)):
        try:
            num = int(value)
            return num if num in (0,1) else None
        except Exception:
            return None
    return None

# 測試normalize_binary_cell
test_values = [
    None,               # 測試 None 值
    float('nan'),       # 測試 NaN 值
    "",                 # 測試空字符串
    "0",                # 測試字符串 "0"
    "1",                # 測試字符串 "1"
    "  1  ",            # 測試帶空格的字符串 "1"
    "2",                # 測試無效的字符串 "2"
    "abc",              # 測試無效的字符串 "abc"
    0,                  # 測試整數 0
    1,                  # 測試整數 1
    2,                  # 測試無效的整數 2
    0.0,                # 測試浮點數 0.0
    1.0,                # 測試浮點數 1.0
    2.0,                # 測試無效的浮點數 2.0
]

# 遍歷測試值並打印結果
for value in test_values:
    result = normalize_binary_cell(value)
    print(f"normalize_binary_cell({value!r}) -> {result}")

normalize_binary_cell(None) -> None
normalize_binary_cell(nan) -> None
normalize_binary_cell('') -> None
normalize_binary_cell('0') -> 0
normalize_binary_cell('1') -> 1
normalize_binary_cell('  1  ') -> 1
normalize_binary_cell('2') -> None
normalize_binary_cell('abc') -> None
normalize_binary_cell(0) -> 0
normalize_binary_cell(1) -> 1
normalize_binary_cell(2) -> None
normalize_binary_cell(0.0) -> 0
normalize_binary_cell(1.0) -> 1
normalize_binary_cell(2.0) -> None


# parse_ops
### 函數目標
`parse_ops` 函數的目標是解析操作字串，並將其轉換為包含操作類型和參數的字典列表。該函數主要用於處理格式化的操作字串，支持以逗號分隔的多個操作（例如 `"W1,W0,W1"`）。

### 函數輸入
- `ops_str`：可以是任何類型的輸入值，包括 `None`、浮點數、字串等。
        - 如果輸入為 `None` 或空字串，返回空列表。
        - 如果輸入為浮點數且為 `NaN`，返回空列表。
        - 如果輸入為格式化的操作字串（例如 `"W1,W0,W1"`），則解析並返回對應的字典列表。

### 函數輸出
- 返回一個字典列表，每個字典包含以下鍵值：
        - `"op"`：操作類型，例如 `"W"``"R"`。
        - `"val"`：操作值，為整數（例如 `1` 或 `0`）。

### 執行過程
1. **檢查空值**：
        - 如果輸入為 `None` 或浮點數且為 `NaN`，返回空列表。
        - 如果輸入為空字串，返回空列表。

2. **處理字串輸入**：
        - 去除字串首尾的空格。
        - 如果字串為空，返回空列表。
        - 將字串中的逗號替換為空格，並使用正則表達式匹配操作模式。
        - 遍歷匹配的操作，提取操作類型和操作值，並將其轉換為字典。

3. **返回結果**：
        - 返回包含所有解析結果的字典列表。

### 測試數據
以下是測試數據及其對應的輸出：
- `'W1 W0 W1'` → `[{'op': 'W', 'val': 1}, {'op': 'W', 'val': 0}, {'op': 'W', 'val': 1}]`
- `'W1,W0,R1'` → `[{'op': 'W', 'val': 1}, {'op': 'W', 'val': 0}, {'op': 'R', 'val': 1}]`
- `'W1, R0, W1'` → `[{'op': 'W', 'val': 1}, {'op': 'R', 'val': 0}, {'op': 'W', 'val': 1}]`
- `'W1'` → `[{'op': 'W', 'val': 1}]`
- `''` → `[]`
- `None` → `[]`

In [19]:
def parse_ops(ops_str: Any) -> List[Dict[str, Any]]:
    """解析操作字串，支援以逗號分隔的多個操作 (例如 'W1,W0,W1')，並包含 R0 和 R1"""
    if ops_str is None:
        return []
    if isinstance(ops_str, float):
        try:
            import math
            if math.isnan(ops_str):
                return []
        except Exception:
            pass
    if isinstance(ops_str, str):
        s = ops_str.strip()
        if s == "":
            return []
        # 修改以支援逗號分隔的多個操作，並包含 R0 和 R1
        ops = []
        for match in re.finditer(r"([WRwr])\s*([01])", s.replace(",", " ")):
            op = match.group(1).upper()
            val = int(match.group(2))
            ops.append({"op": op, "val": val})
        return ops
    return []

#測試parse_ops
test_ops = [
    "W1 W0 W1",        # 測試空格分隔的多個操作
    "W1,W0,R1",        # 測試逗號分隔的多個操作
    "W1, R0, W1",      # 測試逗號和空格混合分隔的多個操作
    "W1",              # 測試單一操作
    "",                # 測試空字符串
    None,              # 測試 None 值
]

# 遍歷測試值並打印結果
for ops in test_ops:
    result = parse_ops(ops)
    print(f"parse_ops({ops!r}) -> {result}")

parse_ops('W1 W0 W1') -> [{'op': 'W', 'val': 1}, {'op': 'W', 'val': 0}, {'op': 'W', 'val': 1}]
parse_ops('W1,W0,R1') -> [{'op': 'W', 'val': 1}, {'op': 'W', 'val': 0}, {'op': 'R', 'val': 1}]
parse_ops('W1, R0, W1') -> [{'op': 'W', 'val': 1}, {'op': 'R', 'val': 0}, {'op': 'W', 'val': 1}]
parse_ops('W1') -> [{'op': 'W', 'val': 1}]
parse_ops('') -> []
parse_ops(None) -> []


# parse_C_pattern
```markdown
### 函數目標
`parse_C_pattern` 函數的目標是解析特定格式的操作字串，並將其轉換為包含操作類型和參數的字典。該函數主要用於處理形如 `C(X)(X)(X)` 的操作字串，將其標準化為結構化數據。

### 函數輸入
- `token`：一個字串，表示操作的原始輸入。
    - 如果輸入為 `None` 或空字串，返回 `None`。
    - 如果輸入不符合格式，返回 `None`。
    - 如果輸入符合格式（例如 `"C(1)(0)(X)"`），則解析並返回對應的字典。

### 函數輸出
- 返回一個字典，包含以下鍵值：
    - `"op"`：操作類型，固定為 `"C"`。
    - `"l"`：左參數，為整數 `0` 或 `1`，或者 `None`（表示 `X`）。
    - `"m"`：中參數，為整數 `0` 或 `1`，或者 `None`（表示 `X`）。
    - `"r"`：右參數，為整數 `0` 或 `1`，或者 `None`（表示 `X`）。
- 如果輸入無效或無法解析，返回 `None`。

### 執行過程
1. **正則匹配**：
    - 使用正則表達式匹配形如 `C(X)(X)(X)` 的字串。
    - 如果匹配失敗，返回 `None`。

2. **參數映射**：
    - 提取匹配結果中的三個參數（左、中、右）。
    - 將參數字元轉換為對應的值：
        - 如果參數為 `"X"`，轉換為 `None`。
        - 如果參數為 `"0"` 或 `"1"`，轉換為整數 `0` 或 `1`。

3. **構造字典**：
    - 將操作類型設置為 `"C"`。
    - 將左、中、右參數分別設置為對應的值。
    - 返回構造的字典。

### 測試數據
以下是測試數據及其對應的輸出：
- `"C(1)(0)(X)"` → `{"op": "C", "l": 1, "m": 0, "r": None}`
- `"C(X)(X)(X)"` → `{"op": "C", "l": None, "m": None, "r": None}`
- `"C(0)(1)(0)"` → `{"op": "C", "l": 0, "m": 1, "r": 0}`
- `"invalid"` → `None`
- `None` → `None`
- `""` → `None`
```

In [6]:
def parse_C_pattern(token: str) -> Optional[Dict[str, Any]]:
    """(See notebook for full docstring)"""
    m = re.fullmatch(r"\s*[Cc]\s*\(\s*([01Xx])\s*\)\s*\(\s*([01Xx])\s*\)\s*\(\s*([01Xx])\s*\)\s*", token or "")
    if not m: return None
    def map_bit(ch: str) -> Optional[int]:
        ch = ch.upper()
        if ch == "X": return None
        return int(ch)
    return {
        "op": "C",
        "l": map_bit(m.group(1)),
        "m": map_bit(m.group(2)),
        "r": map_bit(m.group(3)),
    }

#測試parse_C_pattern
test_patterns = [
    "C(1)(0)(X)",  # Valid pattern with mixed values
    "C(X)(X)(X)",  # Valid pattern with all 'X'
    "C(0)(1)(0)",  # Valid pattern with binary values
    "invalid",     # Invalid pattern
    None,          # None input
    "",            # Empty string
]

# 遍歷測試值並打印結果
for pattern in test_patterns:
    result = parse_C_pattern(pattern)
    print(f"parse_C_pattern({pattern!r}) -> {result}")

parse_C_pattern('C(1)(0)(X)') -> {'op': 'C', 'l': 1, 'm': 0, 'r': None}
parse_C_pattern('C(X)(X)(X)') -> {'op': 'C', 'l': None, 'm': None, 'r': None}
parse_C_pattern('C(0)(1)(0)') -> {'op': 'C', 'l': 0, 'm': 1, 'r': 0}
parse_C_pattern('invalid') -> None
parse_C_pattern(None) -> None
parse_C_pattern('') -> None


# parse_detect
### 函數目標
`parse_detect` 函數的目標是解析輸入的檢測字串，並將其轉換為包含操作類型和參數的字典列表。該函數主要用於處理格式化的檢測字串，將其標準化為結構化數據。

### 函數輸入
- `detect_str`：可以是任何類型的輸入值，包括 `None`、浮點數、字串等。
    - 如果輸入為 `None` 或空字串，返回空列表。
    - 如果輸入為浮點數且為 `NaN`，返回空列表。
    - 如果輸入為格式化的檢測字串（例如 `"R 1/C(1)(0)(X)"`），則解析並返回對應的字典列表。

### 函數輸出
- 返回一個字典列表，每個字典包含以下鍵值：
    - `"op"`：操作類型，例如 `"R"` 或 `"C"`。
    - `"val"`：操作值，為整數（例如 `1`），僅適用於 `"R"` 操作。
    - `"l"`、`"m"`、`"r"`：分別表示 `"C"` 操作的左、中、右參數，為整數 `0` 或 `1`，或者 `None`（表示 `X`）。

### 執行過程
1. **檢查空值**：
    - 如果輸入為 `None` 或浮點數且為 `NaN`，返回空列表。
    - 如果輸入為空字串，返回空列表。

2. **處理字串輸入**：
    - 去除字串首尾的空格。
    - 將字串按斜線（`/`）分割為多個子字串。
    - 遍歷每個子字串，嘗試匹配以下格式：
        - `"R X"`：使用正則表達式匹配 `"R"` 操作，提取操作值並轉換為整數。
        - `"C(X)(X)(X)"`：使用 `parse_C_pattern` 函數解析 `"C"` 操作，提取左、中、右參數。

3. **構造結果**：
    - 將每個成功解析的操作轉換為字典，並加入結果列表。
    - 如果子字串無法匹配任何格式，則忽略該子字串。

4. **返回結果**：
    - 返回包含所有解析結果的字典列表。

### 測試數據
```python
以下是測試數據及其對應的輸出：
- `None` → `[]`
- `float('nan')` → `[]`
- `""` → `[]`
- `"R1/C(1)(0)(X)"` → `[{"op": "R", "val": 1}, {"op": "C", "l": 1, "m": 0, "r": None}]`
- `"R 0/C(X)(X)(X)"` → `[{"op": "R", "val": 0}, {"op": "C", "l": None, "m": None, "r": None}]`
- `"R 1 / C(0)(1)(0)"` → `[{"op": "R", "val": 1}, {"op": "C", "l": 0, "m": 1, "r": 0}]`
- `"invalid"` → `[]`
```

In [8]:
def parse_detect(detect_str: Any) -> List[Dict[str, Any]]:
    """(See notebook for full docstring)"""
    if detect_str is None:
        return []
    if isinstance(detect_str, float):
        try:
            import math
            if math.isnan(detect_str):
                return []
        except Exception:
            pass
    out = []
    s = str(detect_str).strip()
    if s == "":
        return []
    for token in s.split("/"):
        t = token.strip()
        m = re.fullmatch(r"([Rr])\s*([01])", t)
        if m:
            out.append({"op":"R","val":int(m.group(2))})
            continue
        c = parse_C_pattern(t)
        if c:
            out.append(c)
            continue
    return out

#測試parse_detect
test_detects = [
    None,                       # 測試 None 值
    float('nan'),               # 測試 NaN 值
    "",                         # 測試空字符串
    "R1/C(1)(0)(X)",           # 測試有效檢測字串 "R 1/C(1)(0)(X)"
    "R 0/C(X)(X)(X)",           # 測試有效檢測字串 "R 0/C(X)(X)(X)"
    "R 1 / C(0)(1)(0)",         # 測試帶空格的有效檢測字串 "R 1 / C(0)(1)(0)"
    "invalid",                  # 測試無效檢測字串 "invalid"
]

# 遍歷測試值並打印結果
for detect in test_detects:
    result = parse_detect(detect)
    print(f"parse_detect({detect!r}) -> {result}")

parse_detect(None) -> []
parse_detect(nan) -> []
parse_detect('') -> []
parse_detect('R1/C(1)(0)(X)') -> [{'op': 'R', 'val': 1}, {'op': 'C', 'l': 1, 'm': 0, 'r': None}]
parse_detect('R 0/C(X)(X)(X)') -> [{'op': 'R', 'val': 0}, {'op': 'C', 'l': None, 'm': None, 'r': None}]
parse_detect('R 1 / C(0)(1)(0)') -> [{'op': 'R', 'val': 1}, {'op': 'C', 'l': 0, 'm': 1, 'r': 0}]
parse_detect('invalid') -> []


# is_coupling_fault
### 函數目標
`is_coupling_fault` 函數的目標是判斷輸入的 `Da` 和 `Ca` 是否表示耦合故障（coupling fault）。該函數主要用於檢查兩個狀態值是否至少有一個不為 `None`，從而確定是否存在耦合故障。

### 函數輸入
- `Da`：一個可選的整數，表示某種狀態值，可以為 `None`。
- `Ca`：一個可選的整數，表示另一種狀態值，可以為 `None`。

### 函數輸出
- 返回一個布林值（`True` 或 `False`）：
    - 如果 `Da` 或 `Ca` 中至少有一個不為 `None`，則返回 `True`。
    - 如果 `Da` 和 `Ca` 均為 `None`，則返回 `False`。

### 執行過程
1. **檢查輸入值**：
    - 如果 `Da` 不為 `None`，則返回 `True`。
    - 如果 `Ca` 不為 `None`，則返回 `True`。
    - 如果 `Da` 和 `Ca` 均為 `None`，則返回 `False`。

2. **返回結果**：
    - 根據上述條件，返回布林值表示是否存在耦合故障。

### 測試數據
以下是測試數據及其對應的輸出：
- `(None, None)` → `False`：兩個輸入均為 `None`。
- `(0, None)` → `True`：`Da` 為 0，`Ca` 為 `None`。
- `(None, 1)` → `True`：`Da` 為 `None`，`Ca` 為 1。
- `(1, 1)` → `True`：兩個輸入均為 1。
- `(0, 0)` → `True`：兩個輸入均為 0。
- `(None, None)` → `False`：再次測試兩個輸入均為 `None` 的情況。

In [9]:
def is_coupling_fault(Da: Optional[int], Ca: Optional[int]) -> bool:
    """(See notebook for full docstring)"""
    return (Da is not None) or (Ca is not None)

#測試is_coupling_fault
test_cases = [
    (None, None),  # Both None
    (0, None),     # Da is 0, Ca is None
    (None, 1),     # Da is None, Ca is 1
    (1, 1),        # Both are 1
    (0, 0),        # Both are 0
    (None, None),  # Both None again
]

# 遍歷測試值並打印結果
for Da, Ca in test_cases:
    result = is_coupling_fault(Da, Ca)
    print(f"is_coupling_fault(Da={Da}, Ca={Ca}) -> {result}")

is_coupling_fault(Da=None, Ca=None) -> False
is_coupling_fault(Da=0, Ca=None) -> True
is_coupling_fault(Da=None, Ca=1) -> True
is_coupling_fault(Da=1, Ca=1) -> True
is_coupling_fault(Da=0, Ca=0) -> True
is_coupling_fault(Da=None, Ca=None) -> False


# expand_av_relations
```markdown
### 函數目標
`expand_av_relations` 函數的目標是根據輸入的基礎原始資料（`base_primitive`）和是否為耦合故障（`coupling`），生成一個包含 `av_relation` 的擴展資料列表。該函數主要用於處理耦合故障的情況，將其分解為兩種可能的關係（`a<v` 和 `a>v`），或者在非耦合故障的情況下保留原始資料。

### 函數輸入
- `base_primitive`：一個字典，表示基礎的原始資料，包含故障的基本信息。
- `coupling`：一個布林值，表示是否為耦合故障。
    - 如果為 `True`，表示存在耦合故障，需生成兩種可能的 `av_relation`。
    - 如果為 `False`，表示不存在耦合故障，`av_relation` 設置為 `None`。

### 函數輸出
- 返回一個字典列表，每個字典表示擴展後的資料：
    - 如果 `coupling` 為 `True`，返回兩個字典，分別對應 `av_relation` 為 `a<v` 和 `a>v` 的情況。
    - 如果 `coupling` 為 `False`，返回一個字典，`av_relation` 設置為 `None`。

### 執行過程
1. **檢查是否為耦合故障**：
    - 如果 `coupling` 為 `True`，表示存在耦合故障：
        - 使用 `deepcopy` 複製 `base_primitive`，生成兩個副本。
        - 將第一個副本的 `av_relation` 設置為 `a<v`。
        - 將第二個副本的 `av_relation` 設置為 `a>v`。
        - 返回包含這兩個副本的列表。
    - 如果 `coupling` 為 `False`，表示不存在耦合故障：
        - 將 `base_primitive` 的 `av_relation` 設置為 `None`。
        - 返回包含該字典的列表。

### 測試數據
以下是測試數據及其對應的輸出：
- `base_primitive = {"name": "test", "state": {"Da": 1, "Ca": 0}}`, `coupling = True`：
    - 輸出：
      ```python
      [
          {"name": "test", "state": {"Da": 1, "Ca": 0}, "av_relation": "a<v"},
          {"name": "test", "state": {"Da": 1, "Ca": 0}, "av_relation": "a>v"}
      ]
      ```
- `base_primitive = {"name": "test", "state": {"Da": 0, "Ca": None}}`, `coupling = False`：
    - 輸出：
      ```python
      [
          {"name": "test", "state": {"Da": 0, "Ca": None}, "av_relation": None}
      ]
      ```
```

In [10]:
def expand_av_relations(base_primitive: Dict[str, Any], coupling: bool) -> List[Dict[str, Any]]:
    """(See notebook for full docstring)"""
    from copy import deepcopy
    if coupling:
        left = deepcopy(base_primitive); left["av_relation"] = "a<v"
        right = deepcopy(base_primitive); right["av_relation"] = "a>v"
        return [left, right]
    else:
        base_primitive["av_relation"] = None
        return [base_primitive]

#測試expand_av_relations
test_base_primitive = {"name": "test", "state": {"Da": 1, "Ca": 0}}
test_coupling_cases = [True, False]

# 遍歷測試值並打印結果
for coupling in test_coupling_cases:
    result = expand_av_relations(test_base_primitive, coupling)
    print(f"expand_av_relations({test_base_primitive}, {coupling}) -> {result}")

expand_av_relations({'name': 'test', 'state': {'Da': 1, 'Ca': 0}}, True) -> [{'name': 'test', 'state': {'Da': 1, 'Ca': 0}, 'av_relation': 'a<v'}, {'name': 'test', 'state': {'Da': 1, 'Ca': 0}, 'av_relation': 'a>v'}]
expand_av_relations({'name': 'test', 'state': {'Da': 1, 'Ca': 0}, 'av_relation': None}, False) -> [{'name': 'test', 'state': {'Da': 1, 'Ca': 0}, 'av_relation': None}]


# transform_row_to_primitives
### 函數目標
`transform_row_to_primitives` 函數的目標是將輸入的資料行（`pd.Series`）轉換為一個包含多個故障原始資料（primitives）的字典列表。該函數主要用於解析 Excel 資料表中的每一行，並將其標準化為結構化的 JSON 格式。

### 函數輸入
- `row`：一個 `pd.Series` 對象，表示 Excel 資料表中的一行數據。該行應包含以下欄位：
    - `"Name"`：故障名稱，字符串類型。
    - `"Da"`、`"Dv"`、`"Ca"`、`"Cv"`：狀態值，可能為二進制值（0 或 1）、空值或其他類型。
    - `"Ops"`：操作序列，字符串類型。
    - `"Detect"`：檢測條件，字符串類型。

### 函數輸出
- 返回一個字典列表，每個字典表示一個故障原始資料，包含以下鍵值：
    - `"name"`：故障名稱，字符串類型。
    - `"state"`：一個字典，包含 `"Da"`、`"Dv"`、`"Ca"`、`"Cv"` 的狀態值。
    - `"ops_seq"`：操作序列，解析後的字典列表。
    - `"detect_any_of"`：檢測條件，解析後的字典列表。
    - `"av_relation"`：耦合關係，可能為 `"a<v"`、`"a>v"` 或 `None`。
    - `"raw"`：原始輸入數據，包含 `"Ops"` 和 `"Detect"` 的原始字符串。

### 執行過程
1. **提取欄位數據**：
     - 從輸入的 `row` 中提取 `"Name"`、`"Da"`、`"Dv"`、`"Ca"`、`"Cv"`、`"Ops"` 和 `"Detect"` 欄位的值。
     - 使用 `normalize_binary_cell` 函數將 `"Da"`、`"Dv"`、`"Ca"`、`"Cv"` 的值標準化為二進制值（0 或 1）或 `None`。
     - 使用 `parse_ops` 函數解析 `"Ops"` 的操作序列。
     - 使用 `parse_detect` 函數解析 `"Detect"` 的檢測條件。

2. **構造基礎字典**：
     - 創建一個包含故障名稱、狀態值、操作序列、檢測條件和原始數據的基礎字典。

3. **判斷耦合故障**：
     - 使用 `is_coupling_fault` 函數判斷是否存在耦合故障（`Da` 或 `Ca` 至少有一個不為 `None`）。

4. **擴展耦合關係**：
     - 如果存在耦合故障，使用 `expand_av_relations` 函數生成兩個擴展字典，分別對應 `"a<v"` 和 `"a>v"` 的情況。
     - 如果不存在耦合故障，將基礎字典的 `"av_relation"` 設置為 `None`。

5. **返回結果**：
     - 返回包含所有擴展字典的列表。

### 測試數據
以下是測試數據及其對應的輸出：
- 測試數據：
    ```python
    row = pd.Series({
        "Name": "test_fault",
        "Da": 1,
        "Dv": None,
        "Ca": 0,
        "Cv": None,
        "Ops": "W 1, R1",
        "Detect": "R1/C(1)(0)(X)"
    })
    ```
- 預期輸出：
    ```python
    [
        {
            "name": "test_fault",
            "state": {"Da": 1, "Dv": None, "Ca": 0, "Cv": None},
            "ops_seq": [{"op": "W", "val": 1}, {"op": "R", "val": 1}],
            "detect_any_of": [
                {"op": "R", "val": 1},
                {"op": "C", "l": 1, "m": 0, "r": None}
            ],
            "av_relation": "a<v",
            "raw": {"Ops": "W 1", "Detect": "R1/C(1)(0)(X)"}
        },
        {
            "name": "test_fault",
            "state": {"Da": 1, "Dv": None, "Ca": 0, "Cv": None},
            "ops_seq": [{"op": "W", "val": 1}, {"op": "R", "val": 1}],
            "detect_any_of": [
                {"op": "R", "val": 1},
                {"op": "C", "l": 1, "m": 0, "r": None}
            ],
            "av_relation": "a>v",
            "raw": {"Ops": "W 1", "Detect": "R1/C(1)(0)(X)"}
        }
    ]
    ```

In [20]:
def transform_row_to_primitives(row: "pd.Series") -> List[Dict[str, Any]]:
    """(See notebook for full docstring)"""
    name = str(row.get("Name","")).strip()
    Da = normalize_binary_cell(row.get("Da"))
    Dv = normalize_binary_cell(row.get("Dv"))
    Ca = normalize_binary_cell(row.get("Ca"))
    Cv = normalize_binary_cell(row.get("Cv"))
    ops_seq = parse_ops(row.get("Ops"))
    detect_any_of = parse_detect(row.get("Detect"))
    base = {
        "name": name,
        "state": {"Da": Da, "Dv": Dv, "Ca": Ca, "Cv": Cv},
        "ops_seq": ops_seq,
        "detect_any_of": detect_any_of,
        "av_relation": None,
        "raw": {
            "Ops": "" if row.get("Ops") is None else str(row.get("Ops")),
            "Detect": "" if row.get("Detect") is None else str(row.get("Detect")),
        },
    }
    coupling = is_coupling_fault(Da, Ca)
    return expand_av_relations(base, coupling)

#測試transform_row_to_primitives
test_row = pd.Series({
    "Name": "test_fault",
    "Da": 1,
    "Dv": None,
    "Ca": 0,
    "Cv": None,
    "Ops": "W 1,R1",
    "Detect": "R1/C(1)(0)(X)"
})

# 測試transform_row_to_primitives
result = transform_row_to_primitives(test_row)
for primitive in result:
    print(primitive)

{'name': 'test_fault', 'state': {'Da': 1, 'Dv': None, 'Ca': 0, 'Cv': None}, 'ops_seq': [{'op': 'W', 'val': 1}, {'op': 'R', 'val': 1}], 'detect_any_of': [{'op': 'R', 'val': 1}, {'op': 'C', 'l': 1, 'm': 0, 'r': None}], 'av_relation': 'a<v', 'raw': {'Ops': 'W 1,R1', 'Detect': 'R1/C(1)(0)(X)'}}
{'name': 'test_fault', 'state': {'Da': 1, 'Dv': None, 'Ca': 0, 'Cv': None}, 'ops_seq': [{'op': 'W', 'val': 1}, {'op': 'R', 'val': 1}], 'detect_any_of': [{'op': 'R', 'val': 1}, {'op': 'C', 'l': 1, 'm': 0, 'r': None}], 'av_relation': 'a>v', 'raw': {'Ops': 'W 1,R1', 'Detect': 'R1/C(1)(0)(X)'}}


# convert_excel_to_json
### 函數目標
`convert_excel_to_json` 函數的目標是將 Excel 文件中的數據轉換為 JSON 格式，並保存到指定的輸出文件中。該函數主要用於處理包含故障原始資料的 Excel 文件，並生成結構化的 JSON 文件以便進一步分析。

### 函數輸入
- `input_xlsx`：一個字符串，表示輸入的 Excel 文件路徑。
- `output_json`：一個字符串，表示輸出的 JSON 文件路徑。
- `sheet_name`（可選）：一個字符串，表示需要處理的 Excel 工作表名稱。如果未提供，則默認處理第一個工作表。

### 函數輸出
- 返回一個字典，包含以下鍵值：
    - `"schema_version"`：一個字符串，表示 JSON 文件的結構版本。
    - `"source"`：一個字符串，表示輸入的 Excel 文件名。
    - `"fault_primitives"`：一個列表，包含從 Excel 文件中提取並轉換的故障原始資料。

### 執行過程
1. **讀取 Excel 文件**：
    - 如果提供了 `sheet_name`，則讀取指定的工作表。
    - 如果未提供 `sheet_name`，則讀取第一個工作表。

2. **初始化結果列表**：
    - 創建一個空列表 `primitives`，用於存儲轉換後的故障原始資料。

3. **處理每一行數據**：
    - 遍歷 Excel 文件中的每一行數據，將其轉換為故障原始資料。
    - 使用 `transform_row_to_primitives` 函數處理每一行，並將結果擴展到 `primitives` 列表中。

4. **構造結果字典**：
    - 創建一個字典，包含 JSON 文件的結構版本、輸入文件名和轉換後的故障原始資料。

5. **保存為 JSON 文件**：
    - 將結果字典保存到指定的輸出 JSON 文件中，使用 UTF-8 編碼並格式化輸出。

6. **返回結果**：
    - 返回包含所有轉換後數據的結果字典。

### 測試數據
以下是測試數據及其對應的輸出：
- 測試數據：
    - `input_xlsx = "fault_data.xlsx"`
    - `output_json = "fault_primitives.json"`
    - `sheet_name = "Sheet1"`
- 預期輸出：
    ```json
    {
        "schema_version": "0.2",
        "source": "fault_data.xlsx",
        "fault_primitives": [
            {
                "name": "test_fault",
                "state": {"Da": 1, "Dv": null, "Ca": 0, "Cv": null},
                "ops_seq": [{"op": "W", "val": 1}, {"op": "R", "val": 1}],
                "detect_any_of": [
                    {"op": "R", "val": 1},
                    {"op": "C", "l": 1, "m": 0, "r": null}
                ],
                "av_relation": "a<v",
                "raw": {"Ops": "W 1,R1", "Detect": "R1/C(1)(0)(X)"}
            },
            {
                "name": "test_fault",
                "state": {"Da": 1, "Dv": null, "Ca": 0, "Cv": null},
                "ops_seq": [{"op": "W", "val": 1}, {"op": "R", "val": 1}],
                "detect_any_of": [
                    {"op": "R", "val": 1},
                    {"op": "C", "l": 1, "m": 0, "r": null}
                ],
                "av_relation": "a>v",
                "raw": {"Ops": "W 1,R1", "Detect": "R1/C(1)(0)(X)"}
            }
        ]
    }
    ```

In [21]:
def convert_excel_to_json(input_xlsx: str, output_json: str, sheet_name: Optional[str] = None) -> Dict[str, Any]:
    """(See notebook for full docstring)"""
    if sheet_name is not None:
        df = pd.read_excel(input_xlsx, sheet_name=sheet_name)
    else:
        df = pd.read_excel(input_xlsx)
    primitives: List[Dict[str, Any]] = []
    for _, row in df.iterrows():
        primitives.extend(transform_row_to_primitives(row))
    result = {
        "schema_version": "0.2",
        "source": os.path.basename(input_xlsx),
        "fault_primitives": primitives,
    }
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    return result

#測試convert_excel_to_json
test_input_xlsx = "test_fault_data.xlsx"
test_output_json = "test_fault_primitives.json"

# Create a sample DataFrame to simulate the Excel file
sample_data = {
    "Name": ["test_fault"],
    "Da": [1],
    "Dv": [None],
    "Ca": [0],
    "Cv": [None],
    "Ops": ["W 1,R1"],
    "Detect": ["R1/C(1)(0)(X)"],
}
df = pd.DataFrame(sample_data)
df.to_excel(test_input_xlsx, index=False)

# Run the function and print the result
result = convert_excel_to_json(test_input_xlsx, test_output_json)
print(json.dumps(result, ensure_ascii=False, indent=2))

# Clean up the test files
os.remove(test_input_xlsx)
os.remove(test_output_json)

{
  "schema_version": "0.2",
  "source": "test_fault_data.xlsx",
  "fault_primitives": [
    {
      "name": "test_fault",
      "state": {
        "Da": 1,
        "Dv": null,
        "Ca": 0,
        "Cv": null
      },
      "ops_seq": [
        {
          "op": "W",
          "val": 1
        },
        {
          "op": "R",
          "val": 1
        }
      ],
      "detect_any_of": [
        {
          "op": "R",
          "val": 1
        },
        {
          "op": "C",
          "l": 1,
          "m": 0,
          "r": null
        }
      ],
      "av_relation": "a<v",
      "raw": {
        "Ops": "W 1,R1",
        "Detect": "R1/C(1)(0)(X)"
      }
    },
    {
      "name": "test_fault",
      "state": {
        "Da": 1,
        "Dv": null,
        "Ca": 0,
        "Cv": null
      },
      "ops_seq": [
        {
          "op": "W",
          "val": 1
        },
        {
          "op": "R",
          "val": 1
        }
      ],
      "detect_any_of": [
        {
   

In [23]:
def main() -> int:
    # 在這裡直接設定路徑
    input_xlsx = "../fault_with_S.xlsx"  # 輸入 Excel 路徑
    output_json = "../fault_primitives.json"  # 輸出 JSON 路徑
    sheet = None  # 如果需要特定工作表名稱，請在此處設定，例如 "Sheet1"

    if not os.path.exists(input_xlsx):
        print(f"Input Excel not found: {input_xlsx}")
        return 2

    res = convert_excel_to_json(input_xlsx, output_json, sheet)
    print(f"Wrote {len(res['fault_primitives'])} primitives to {output_json}")
    return 0


if __name__ == "__main__":
    raise SystemExit(main())

Wrote 52 primitives to ../fault_primitives.json


SystemExit: 0

/home/bb232399/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
